In [1]:
import torch
import torchvision

print(torch.cuda.is_available())
print(torch.__version__)
print(torchvision.__version__)

True
1.10.2+cu113
0.11.3+cu113


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch
from tqdm import tqdm
from torch.nn import functional as F
import types
import os
import MoCo.config as config
from MoCo.dataset import custom_dataset  # utils 파일 -> autoaugment를 MoCo.autoaugment로 변경
from MoCo.utils import get_transform, dataset_info
from MoCo.wideresnet import WideResNet

## utils

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def get_shuffle_idx(bs, device):  # bs=batch size
    """shuffle index for ShuffleBN """
    shuffle_value = torch.randperm(bs).long().to(device)  # index 2 value / bs만큼 랜덤하게 index 추출
    reverse_idx = torch.zeros(bs).long().to(device)  # [0] * bs
    arange_index = torch.arange(bs).long().to(device)  # [0, 1, ..., bs-1]
    reverse_idx.index_copy_(0, shuffle_value, arange_index)  # value back to index
    
    return shuffle_value, reverse_idx

In [5]:
bs = 10
device = config.DEVICE

shuffle_value = torch.randperm(bs).long().to(device)  # index 2 value / bs만큼 랜덤하게 index 추출
reverse_idx = torch.zeros(bs).long().to(device)  # [0] * bs
arange_index = torch.arange(bs).long().to(device)  # [0, 1, ..., bs-1]

print('shuffle_value : ', shuffle_value)
print('reverse_idx : ', reverse_idx)
print('arange_index :', arange_index)
print()

print(reverse_idx.index_copy_(0, shuffle_value, arange_index))  # value back to index
print(reverse_idx)
print(shuffle_value)

shuffle_value :  tensor([5, 1, 9, 3, 4, 8, 7, 0, 2, 6], device='cuda:0')
reverse_idx :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
arange_index : tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0')

tensor([7, 1, 8, 3, 4, 0, 9, 6, 5, 2], device='cuda:0')
tensor([7, 1, 8, 3, 4, 0, 9, 6, 5, 2], device='cuda:0')
tensor([5, 1, 9, 3, 4, 8, 7, 0, 2, 6], device='cuda:0')


In [6]:
def enqueue(queue, k):
    return torch.cat([queue, k], dim=0)  # 현재의 mini-batch 추가 

def dequeue(queue, max_len=config.QUEUE_LENGTH):  # maxlen=4096
    if queue.shape[0] >= max_len:
        return queue[-max_len:]  # queue follows FIFO (가장 과거의 mini-batch를 삭제)
    else:
        return queue

## Data

In [7]:
image_size, mean, std = dataset_info(name='cifar')
print(image_size)
print(mean)
print(std)

32
(0.4914, 0.4822, 0.4465)
(0.2471, 0.2435, 0.2616)


In [8]:
train_transform = get_transform(image_size, mean=mean, std=std, mode='train')

train_dataset = custom_dataset(datasets.cifar.CIFAR10)(root='./', train=True, transform=train_transform,  # datasets.mnist.MNIST
                                                       download=True)

train_dataloader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0,
                              pin_memory=False, drop_last=True)  # drop the last batch due to irregular size

print(len(train_dataset))

Files already downloaded and verified
50000


## Query & Key encoder

In [34]:
def get_model(model_name='WideResNet'):
    
    # model = CustomNetwork (WideResNet class의 객체 instance)
    model_q = WideResNet(pretrained=None)  
    model_k = WideResNet(pretrained=None)

    def forward(self, input):
        x = self.features(input)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = F.normalize(x)  # l2 normalize by default

        return x

    model_q.forward = types.MethodType(forward, model_q)  # 객체 instance에 forward mehod 추가
    model_k.forward = types.MethodType(forward, model_k)

    # for model k, it doesn't require grad
    for param in model_k.parameters():
        param.requires_grad = False

    device_list = [config.GPU_ID] # Shuffle BN can be applied through there is only one gpu.

    model_q = torch.nn.DataParallel(model_q, device_ids=device_list)
    model_k = torch.nn.DataParallel(model_k, device_ids=device_list)

    model_q.to(config.DEVICE)
    model_k.to(config.DEVICE)

    return model_q, model_k

In [35]:
model_q, model_k = get_model(model_name='WideResNet')  # N개의 batch sample을 각각 128차원의 representation으로 encoding

In [36]:
model_q.forward

<bound method DataParallel.forward of DataParallel(
  (module): WideResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (block1): NetworkBlock(
      (layer): Sequential(
        (0): BasicBlock(
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (convShortcut): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BasicBlock(
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(32, 32

## Train

In [37]:
# Key encoder 갱신
def momentum_update(model_q, model_k, m=0.999):
    """ model_k = m * model_k + (1 - m) model_q """
    for p1, p2 in zip(model_q.parameters(), model_k.parameters()):
        p2.data.mul_(m).add_(1 - m, p1.detach().data)

In [38]:
# query encoder만 역전파를 통해 갱신
optimizer = torch.optim.SGD(model_q.parameters(), lr=0.02, momentum=0.9, nesterov=True, weight_decay=1e-5)  
per = config.ALL_EPOCHS // 6
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[per * 2, per * 4, per * 5], gamma=0.1)

# copy parameters from model_q to model_k (initialize)
momentum_update(model_q, model_k, 0)

criterion = torch.nn.CrossEntropyLoss()

In [39]:
queue = None  # negative sample 저장

torch.backends.cudnn.benchmark = True
start_epoch = 0
min_loss = float('inf')

In [40]:
def train(train_dataloader, model_q, model_k, queue, optimizer, device, t=0.07):
    
    model_q.train()
    model_k.train()
    
    losses = AverageMeter()
    pred_meter = AverageMeter()
    
    for i, (img_q, img_k, _) in enumerate(tqdm(train_dataloader)):
        if queue is not None and queue.shape[0] == config.QUEUE_LENGTH:
            # queries & (matching) keys
            img_q, img_k = img_q.to(device), img_k.to(device)
            
            # query features
            q = model_q(img_q)  # N x C (C=128)

            # shuffle BN
            shuffle_idx, reverse_idx = get_shuffle_idx(config.BATCH_SIZE, device)  # N=256
            img_k = img_k[shuffle_idx]
            
            # key features
            k = model_k(img_k)  # N x C
            k = k[reverse_idx].detach()  # reverse and no graident to key

            N, C = q.shape
            # K = config.QUEUE_LENGTH

            l_pos = torch.bmm(q.view(N, 1, C), k.view(N, C, 1)).view(N, 1)  # positive logit N x 1 (query & key)
            l_neg = torch.mm(q.view(N, C), queue.transpose(0, 1))  # negative logit N x K (query & queue)

            labels = torch.zeros(N, dtype=torch.long).to(device)  # positives are the 0-th
            logits = torch.cat([l_pos, l_neg], dim=1) / t  # Nx(K+1)
            # print(logits[0])
            
            pred = logits[:, 0].mean()
            loss = criterion(logits, labels)
            
            losses.update(loss.item(), N)
            pred_meter.update(pred.item(), N)

            # update model_q (back-propagation)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update model_k by momentum (no gradients)
            momentum_update(model_q, model_k, 0.999)

        else:
            img_k = img_k.to(device)
            shuffle_idx, reverse_idx = get_shuffle_idx(config.BATCH_SIZE, device)
            img_k = img_k[shuffle_idx]
            k = model_k(img_k)  # N x C
            k = k[reverse_idx].detach()  # reverse and no graident to key

        # update dictionary
        queue = enqueue(queue, k) if queue is not None else k  # key 추가
        queue = dequeue(queue)

    return {'loss': losses.avg,
             'pred': pred_meter.avg}, queue


In [41]:
save_path = './models/MoCov1_weights2.pt'  # 학습된 가중치 저장

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSerror:
        print('Error')
        
createFolder('./models')

In [42]:
for epoch in range(start_epoch, config.ALL_EPOCHS):
    ret, queue = train(train_dataloader, model_q, model_k, queue, optimizer, config.DEVICE)
    ret_str = ' - '.join([f'{k}:{v:.4f}' for k, v in ret.items()])
    print(f'epoch:{epoch} {ret_str}')
    scheduler.step()
    # print(type(config))
    
    if ret['loss'] < min_loss:
        min_loss = ret['loss']
        
        # weights.pt에 저장되는 사항들
        state = {
            # 'config': config.__dict__,
            'model_q': model_q.module.state_dict(),
            'model_k': model_k.module.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'epoch': epoch,
            'min_loss': min_loss
            }
        
        print(f'save to {save_path }')
        torch.save(state, save_path )

100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [00:59<00:00,  3.30it/s]


epoch:0 loss:7.9926 - pred:5.1171
save to ./models/MoCov1_weights2.pt


100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [00:59<00:00,  3.27it/s]


epoch:1 loss:7.6704 - pred:2.6808
save to ./models/MoCov1_weights2.pt


100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [00:59<00:00,  3.26it/s]


...

epoch:177 loss:0.7510 - pred:11.7147


100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [00:59<00:00,  3.28it/s]


epoch:178 loss:0.7572 - pred:11.7017


100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [00:59<00:00,  3.28it/s]

epoch:179 loss:0.7540 - pred:11.7082
